In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("./dataset/DM1_game_dataset.csv")

 deleating the numcomments column

In [ ]:
df = df.drop(labels = "NumComments", axis = 1)

understanding commaxplaytime and mfgplaytime

In [ ]:
df[df["MfgPlaytime"] == df["ComMaxPlaytime"]]["BGGId"].count()

np.int64(21925)

In [33]:
df["ComMaxPlaytime"].sort_values(ascending = False)[0:10]

4349     60000
19044    22500
1713     17280
15642    14400
8120     12000
18374    12000
8052     10000
2059      8640
14258     7920
8503      6000
Name: ComMaxPlaytime, dtype: int64

In [31]:
df["ComMaxPlaytime"].mean()

np.float64(90.51352337514253)

In [32]:
df["ComMaxPlaytime"].median()

45.0

In [34]:
#decided to drop MfgPlaytime
df = df.drop(labels = "MfgPlaytime", axis = 1)

In [46]:
df[(df["IsReimplementation"] == 0) & (df["NumImplementations"] > 0)].loc[:]["IsReimplementation"] = 1

In [45]:
df[(df["IsReimplementation"] == 0) & (df["NumImplementations"] > 0)]

,BGGId,Name,Description,YearPublished,GameWeight,ComWeight,MinPlayers,MaxPlayers,ComAgeRec,LanguageEase,...,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens,Rating
6,25841,Le Grand Empire,le grand empire simulate napoleonic war st...,2007,3.7000,3.8964,2,7,NaN,388.000000,...,21926,0,0,1,0,0,0,0,0,High
8,8983,I.Q. 2000,children trivium game player outward solar sys...,1984,1.6667,1.9372,2,6,8.000000,19.666667,...,21926,0,0,0,0,0,0,0,0,Low
15,16432,Vinegar Joe's War: CBI,vinegar joes war cbi chinaburmaindia theater o...,2005,3.3750,3.4978,1,2,NaN,NaN,...,21926,0,0,1,0,0,0,0,0,Medium
35,75782,The Phantom League,phantom league captain merchant spaceship begi...,2010,3.2222,3.4487,2,6,11.333333,133.000000,...,21926,1,0,0,0,0,0,0,0,Medium
42,130997,Flames of War: Open Fire!,flame war world war ii tabletop game use army ...,2012,3.7500,4.0039,2,2,14.000000,NaN,...,21926,0,0,0,0,0,0,0,0,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21890,1608,Ambush!,boxambush solitaire game combat adventure hero...,1983,3.2491,3.5067,1,1,13.071429,44.166667,...,21926,0,0,1,0,0,0,0,0,High
21895,57834,Counter-Attack! Arras,publish battle magazine counterattack arra dep...,2009,2.3000,2.5666,2,2,14.000000,22.666667,...,21926,0,0,1,0,0,0,0,0,Low
21903,64331,Bar-Lev: The 1973 Arab-Israeli War,quotbarlev arabisraeli warquot wargame ara...,1977,3.8333,4.0064,2,2,16.000000,18.000000,...,21926,0,0,1,0,0,0,0,0,High
21911,638,Hera and Zeus,player oppose god greek mythology hera zeus be...,2000,2.0456,2.3180,2,2,9.846154,16.875000,...,21926,0,1,0,0,0,0,0,0,Medium
